In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import date
from glob import glob
from os.path import join

import branca.colormap as cm
import gpxpy
import folium
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from folium.plugins import HeatMap
from scipy.linalg import norm
from shapely.geometry import Point, Polygon

In [ ]:
import sys

sys.path.insert(1, "../src/")

from mapping import (
    load_gpx,
    update_html_style_and_title,
)

## Plot

In [ ]:
route_path = "../gpx/"
output_path = "../output/"
index_path = "../"

In [ ]:
m = folium.Map(location=[51.2412, -0.5744], zoom_start=11,)

In [ ]:
borough_json = folium.GeoJson(
    "https://findthatpostcode.uk/areas/E07000209.geojson",
    name="geojson",
    style_function=lambda x: {"fillColor": "#00000000", "color": "black", "weight": 3,},
)
borough_json.add_to(m)

In [ ]:
borough_polygon = Polygon(
    np.array(borough_json.data["features"][0]["geometry"]["coordinates"][0])[:, ::-1]
)

In [ ]:
# Add each route
all_routes = sorted(glob(join(route_path, "*.gpx")))
for route in all_routes:
    points = load_gpx(route)
    segments = compute_segments_in_borough(points, borough_polygon)
    for segment in segments:
        folium.PolyLine(segment, color="red", weight=2).add_to(m)

In [ ]:
m